# Decorators

1. Inner Functions
2. Returning Functions from Functions
3. Simple Decorators
4. Syntactic Sugar
5. Reusing Decorators
6. Decorating Functions with Arguments
7. Returning Values From Decorated Functions
8. Who Are You, Really?
9. A Few Real World Examples

By definition, a decorator is a function that takes another function and extends the behavior of the latter function without explicitly modifying it.

This sounds confusing, but it’s really not, especially after you’ve seen a few examples of how decorators work.

In Python, functions are [first-class objects](https://dbader.org/blog/python-first-class-functions). This means that functions can be passed around and used as arguments, just like any other object (string, int, float, list, and so on). Consider the following three functions:

In [1]:
def say_hello(name):
    return f"Hello {name}"

def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"

def greet_bob(greeter_func):
    return greeter_func("Bob")

Here, `say_hello()` and `be_awesome()` are regular functions that expect a name given as a string. The `greet_bob()` function however, expects a function as its argument. We can, for instance, pass it the `say_hello()` or the `be_awesome()` function:

In [2]:
greet_bob(say_hello)

'Hello Bob'

In [3]:
greet_bob(be_awesome)

'Yo Bob, together we are the awesomest!'

## Inner Functions

It’s possible to define functions inside other functions. Such functions are called [inner functions](https://realpython.com/inner-functions-what-are-they-good-for/). Here’s an example of a function with two inner functions:

In [5]:
print("Printing from the parent() function")

def first_child():
    print("Printing from the first_child() function")

def second_child():
    print("Printing from the second_child() function")

second_child()
first_child()

Printing from the parent() function
Printing from the second_child() function
Printing from the first_child() function


Note that the order in which the inner functions are defined does not matter. Like with any other functions, the printing only happens when the inner functions are executed.

Furthermore, the inner functions are not defined until the parent function is called. They are locally scoped to `parent()`: they only exist inside the `parent()` function as local variables. Try calling `first_child()`. You should get an error:

In [6]:
first_child()

Printing from the first_child() function


Whenever you call `parent()`, the inner functions `first_child()` and `second_child()` are also called. But because of their local scope, they aren’t available outside of the `parent()` function.

## Returning Functions From Functions

Python also allows you to use functions as return values. The following example returns one of the inner functions from the outer `parent()` function:

In [7]:
def parent(num):
    def first_child():
        return "Hi, I am Emma"

    def second_child():
        return "Call me Liam"

    if num == 1:
        return first_child
    else:
        return second_child

Note that you are returning `first_child` without the parentheses. Recall that this means that you are returning a reference to the function `first_child`.

## Simple Decorators

Now that you’ve seen that functions are just like any other object in Python, you’re ready to move on and see the magical beast that is the Python decorator. Let’s start with an example:

In [22]:
def my_decorator(func):
    def wrapper():
        print('this is done before calling function')
        func()
        print('this is done after calling function')
        
    return wrapper

def say_weather():
    print("Hi! today is sunny!")
    
# as the output of my_decoration function, function wrapper is well defined since we call it through calling my_decorator
say_weather = my_decorator(say_weather)
say_weather()

# here, wrapper is not defined
wrapper()

this is done before calling function
Hi! today is sunny!
this is done after calling function


NameError: name 'wrapper' is not defined

In [9]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

def say_whee():
    print("Whee!")

In [10]:
say_whee = my_decorator(say_whee)

In [11]:
say_whee()

Something is happening before the function is called.
Whee!
Something is happening after the function is called.


The so-called decoration happens at the following line:

> say_whee = my_decorator(say_whee)

In effect, the name say_whee now points to the `wrapper()` inner function. Remember that you return wrapper as a function when you call `my_decorator(say_whee)`:

In [12]:
say_whee()

Something is happening before the function is called.
Whee!
Something is happening after the function is called.


However, `wrapper()` has a reference to the original `say_whee()` as func, and calls that function between the two calls to `print()`.

Put simply: **decorators wrap a function, modifying its behavior.**

Before moving on, let’s have a look at a second example. Because `wrapper()` is a regular Python function, the way a decorator modifies a function can change dynamically. So as not to disturb your neighbors, the following example will only run the decorated code during the day:

In [13]:
from datetime import datetime

def my_decorator(func):
    def wrapper():
        if datetime.now().hour > 22:
            func()

    return wrapper

def say_hello():
    print("hi")

In [14]:
from datetime import datetime

def not_during_the_night(func):
    def wrapper():
        if 7 <= datetime.now().hour < 22:
            func()
        else:
            pass  # Hush, the neighbors are asleep
    return wrapper

def say_whee():
    print("Whee!")

In [15]:
say_whee = not_during_the_night(say_whee)

If you try to call `say_whee()` after bedtime, nothing will happen:

## Syntactic Sugar!

The way you decorated `say_whee()` above is a little clunky. First of all, you end up typing the name `say_whee` three times. In addition, the decoration gets a bit hidden away below the definition of the function.

Instead, Python allows you to use decorators in a simpler way with the @ symbol, sometimes called the pie syntax (Barry Warsaw named this the 'pie-decorator' syntax, in honor of the Pie-thon Parrot shootout which occurred around the same time as the decorator syntax, and because the @ looks a little like a pie.)

The following example does the exact same thing as the first decorator example:

In [16]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
        
    return wrapper

@my_decorator
def say_whee():
    """
    This function returns some stuff!
    """
    print("Whee!")

So, `@my_decorator` is just an easier way of saying `say_whee = my_decorator(say_whee)`. It’s how you apply a decorator to a function.

## Reusing Decorators

Recall that a decorator is just a regular Python function. All the usual tools for easy reusability are available. Let’s move the decorator to its own module that can be used in many other functions.

Create a file called `decorators.py` with the following content:

In [ ]:
def do_twice(func):
    def wrapper_do_twice():
        func()
        func()
    return wrapper_do_twice

**Note:** You can name your inner function whatever you want, and a generic name like `wrapper()` is usually okay. You’ll see a lot of decorators in this article. To keep them apart, we’ll name the inner function with the same name as the decorator but with a wrapper_ prefix.

You can now use this new decorator in other files by doing a regular import:

```python
from decorators import do_twice

@do_twice
def say_whee():
    print("Whee!")
```

When you run this example, you should see that the original `say_whee()` is executed twice:

In [17]:
@do_twice
def say_whee():
    print("Whee!")

NameError: name 'do_twice' is not defined

In [ ]:
say_whee()

## Decorating Functions with Arguments

Say that you have a function that accepts some arguments. Can you still decorate it? Let’s try:

In [ ]:
@do_twice
def greet(name):
    print(f"Hello {name}")

Unfortunately, running this code raises an error:

In [ ]:
greet("Ali")

The problem is that the inner function `wrapper_do_twice()` does not take any arguments, but name="World" was passed to it. You could fix this by letting `wrapper_do_twice()` accept one argument, but then it would not work for the `say_whee()` function you created earlier.

The solution is to use `*args` and `**kwargs` in the inner wrapper function. Then it will accept an arbitrary number of positional and keyword arguments. Rewrite `decorators.py` as follows:

In [ ]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
        
    return wrapper_do_twice

In [ ]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper_do_twice

@do_twice
def greet(name):
    print(f"Hello {name}")
    
@do_twice
def greet(name):
    print(f"Hello {name}")

The `wrapper_do_twice()` inner function now accepts any number of arguments and passes them on to the function it decorates. Now both your `say_whee()` and `greet()` examples works:

In [ ]:
say_whee()

In [ ]:
greet("Ali")

## Returning Values From Decorated Functions
What happens to the return value of decorated functions? Well, that’s up to the decorator to decide. Let’s say you decorate a simple function as follows:

In [ ]:
@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

In [ ]:
output = return_greeting("Ali")
print(output)

Oops, your decorator ate the return value from the function.

Because the `do_twice_wrapper()` doesn’t explicitly return a value, the call `return_greeting("Ali")` ended up returning None.

To fix this, you need to make sure the wrapper function returns the return value of the decorated function. Change your `decorators.py` file:

In [ ]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        output = func(*args, **kwargs)
        return output
    return wrapper_do_twice

@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

The return value from the last execution of the function is returned:

In [ ]:
output = return_greeting("Ali")
print(output)

## Who Are You, Really?

A great convenience when working with Python, especially in the interactive shell, is its powerful introspection ability. Introspection is the ability of an object to know about its own attributes at runtime. For instance, a function knows its own name and documentation:

In [18]:
print

<function print>

In [19]:
print.__name__

'print'

In [20]:
help(print)

Help on built-in function print in module builtins:

print(...)
    print(value, ..., sep=' ', end='\n', file=sys.stdout, flush=False)
    
    Prints the values to a stream, or to sys.stdout by default.
    Optional keyword arguments:
    file:  a file-like object (stream); defaults to the current sys.stdout.
    sep:   string inserted between values, default a space.
    end:   string appended after the last value, default a newline.
    flush: whether to forcibly flush the stream.



The introspection works for functions you define yourself as well:

In [ ]:
say_whee

In [ ]:
say_whee.__name__

In [ ]:
help(say_whee)

`wrapper_do_twice()`

However, after being decorated, `say_whee()` has gotten very confused about its identity. It now reports being the `wrapper_do_twice()` inner function inside the `do_twice()` decorator. Although technically true, this is not very useful information.

To fix this, decorators should use the `@functools.wraps` decorator, which will preserve information about the original function. Update decorators.py again:

In [ ]:
import functools

def do_twice(func):
    @functools.wraps(func)
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

@do_twice
def say_whee():
    """
    This function says whee!
    """
    print("Whee!")

You do not need to change anything about the decorated `say_whee()` function:

In [ ]:
say_whee

In [ ]:
say_whee.__name__

In [ ]:
help(say_whee)

Much better! Now `say_whee()` is still itself after decoration.

## A Few Real World Examples

Let’s look at a few more useful examples of decorators. You’ll notice that they’ll mainly follow the same pattern that you’ve learned so far:

In [ ]:
import functools

def decorator(func):
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):
        # Do something before
        value = func(*args, **kwargs)
        # Do something after
        return value
    return wrapper_decorator

This formula is a good boilerplate template for building more complex decorators.

## Timing Functions

Let’s start by creating a `@timer` decorator. It will measure the time a function takes to execute and print the duration to the console. Here’s the code:

In [ ]:
import functools
import time

def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])

This decorator works by storing the time just before the function starts running (at the line marked # 1) and just after the function finishes (at # 2). The time the function takes is then the difference between the two (at # 3). We use the `time.perf_counter()` function, which does a good job of measuring time intervals. Here are some examples of timings:

**Note:** The `@timer` decorator is great if you just want to get an idea about the runtime of your functions. If you want to do more precise measurements of code, you should instead consider the `timeit` module in the standard library. It temporarily disables garbage collection and runs multiple trials to strip out noise from quick function calls.

## Debugging Code

The following `@debug` decorator will print the arguments a function is called with as well as its return value every time the function is called:

In [ ]:
def debug(func):
    """Print the function signature and return value"""
    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args]                      # 1
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  # 2
        signature = ", ".join(args_repr + kwargs_repr)           # 3
        print(f"Calling {func.__name__}({signature})")
        value = func(*args, **kwargs)
        print(f"{func.__name__!r} returned {value!r}")           # 4
        return value
    return wrapper_debug

The signature is created by joining the string representations of all the arguments. The numbers in the following list correspond to the numbered comments in the code:

- Create a list of the positional arguments. Use `repr()` to get a nice string representing each argument.
- Create a list of the keyword arguments. The f-string formats each argument as `key=value` where the `!r` specifier means that `repr()` is used to represent the value.
- The lists of positional and keyword arguments is joined together to one signature string with each argument separated by a comma.
- The return value is printed after the function is executed.

Let’s see how the decorator works in practice by applying it to a simple function with one position and one keyword argument:

In [ ]:
@debug
def make_greeting(name, age=None):
    if age is None:
        return f"Howdy {name}!"
    else:
        return f"Whoa {name}! {age} already, you are growing up!"


Note how the `@debug` decorator prints the signature and return value of the `make_greeting()` function:

In [ ]:
make_greeting("Ali")

In [ ]:
make_greeting("Ali", age=28)

This example might not seem immediately useful since the `@debug` decorator just repeats what you just wrote. It’s more powerful when applied to small convenience functions that you don’t call directly yourself.

The following example calculates an approximation to the mathematical constant e:

In [ ]:
import math

# Apply a decorator to a standard library function
math.factorial = debug(math.factorial)

def approximate_e(terms=18):
    return sum(1 / math.factorial(n) for n in range(terms))

This example also shows how you can apply a decorator to a function that has already been defined. The approximation of e is based on the following series expansion:

$e = \sum_{n=0}^{\infty} \frac{1}{n} = \frac{1}{0!} + \frac{1}{1!} + \frac{1}{2!} + \frac{1}{3!} + ...$

When calling the `approximate_e()` function, you can see the `@debug` decorator at work:

In [ ]:
approximate_e(5)

## Slowing Down Code

This next example might not seem very useful. Why would you want to slow down your Python code? Probably the most common use case is that you want to rate-limit a function that continuously checks whether a resource—like a web page—has changed. The `@slow_down` decorator will sleep one second before it calls the decorated function:

In [ ]:
import functools
import time

def slow_down(func):
    """Sleep 1 second before calling the function"""
    @functools.wraps(func)
    def wrapper_slow_down(*args, **kwargs):
        time.sleep(1)
        return func(*args, **kwargs)
    return wrapper_slow_down

@slow_down
def countdown(from_number):
    if from_number < 1:
        print("Liftoff!")
    else:
        print(from_number)
        countdown(from_number - 1)

In [ ]:
countdown(3)

Hello World
function foo
GoodBye World


NameError: name 'wrapper' is not defined